## 获取MNIST数据集和数据探索

In [14]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/home/mj/Documents/data/MNIST_data',one_hot=True)

Extracting /home/mj/Documents/data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/mj/Documents/data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/mj/Documents/data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/mj/Documents/data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
##探索数据
print ('Training data size: ',mnist.train.num_examples)

print ('Validating data size: ',mnist.validation.num_examples)

print ('Testing data size: ',mnist.test.num_examples)

print ('Example training size: ',mnist.train.images[0])

print ('Example training data label:',mnist.train.labels[0])

Training data size:  55000
Validating data size:  5000
Testing data size:  10000
Example training size:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.       

### 获取小的batch

In [3]:
batch_size = 100
xs,ys = mnist.train.next_batch(batch_size)
#从train的集合中获取batch_size大小的训练数据
print ('X shape:',xs.shape)
print ('Y shape:',ys.shape)


X shape: (100, 784)
Y shape: (100, 10)


In [31]:
## 训练模型及不同模型的结果对比
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#MNIST相关常数
INPUT_NODE = 784
OUTPUT_NODE = 10

#配置神经网络参数
LAYER1_NODE = 500 #隐藏层节点数，这里使用一个隐藏层的网络结构数据

BATCH_SIZE = 100 #一个训练batch中的训练数据个数，数字越小时，训练过程越接近

LEARNING_RATE_BASE = 0.8 #基础的学习率
LEARNING_RATE_DECAY = 0.99 #学习率的衰减率

REGULARIZATION_RATE = 0.0001 #描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEPS = 300000
MOVING_AVERAGE_DECAY=0.99 #滑动平均衰减率

def inference(input_tensor,avg_class,weights1,biases1,
             weights2,biases2):
    #当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class == None:
        #计算隐藏层的前向传播结果，这里使用relu激活函数
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
        #计算输出层的前向传播结果，因为在计算损失函数时候会一并计算softmax函数，
        #所以这里不需要加入激活函数，而且不加入softmax不会影响预测结果。因为预测时候
        #使用的是不同类别对应节点输出值的相对大小，有没有softmax层对最后分类结果
        #计算没有影响，于是在计算整个神经网络的前向传播时可以不加入最后的softmax层
        return tf.matmul(layer1,weights2) + biases2
    else:
        #首先使用avg_class.average函数计算得出变量的滑动平均
        layer1 = tf.nn.relu(
            tf.matmul(input_tensor,avg_class.average(weights1)) + avg_class.average(biases1)
        )
        return tf.matmul(layer1,avg_class.average(weights2)) + avg_class.average(biases2)

    #训练过程
def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y-input')
    
    #生成隐藏层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biass1 = tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    
    #生成输出层
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    #前向传播的时候，这里给出的用于计算滑动平均的类为NONE，所以函数不会使用参数的滑动平均
    y = inference(x,None,weights1,biass1,weights2,biases2)
    
    #定义存储训练轮数的变量，这个变量不需要计算滑动平均
    #这里指定这歌变量为不可以训练的变量(trainable=False).
    #在使用TensorFlow训练神经网络时候
    #一般会将代表训练轮数的变量指定为不可训练的参数
    global_step = tf.Variable(0,trainable=False)
    
    #给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类，
    #定训练轮数的变量可以加快训练早期变量的更新速度。
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY,global_step
    )
    
    #在所有代表神经网络参数的变量上使用滑动平均，其他辅助变量(比如global_step)就不需要了
    #获取图上集合的元素
    variables_average_op = variable_averages.apply(tf.trainable_variables())
    
    #计算使用了滑动平均之后的前向传播结果
    average_y = inference(x,variable_averages,
                         weights1,biass1,weights2,biases2)
    #计算交叉熵作为刻画预测值和真实值之间差距的损失函数。
    #交叉熵函数，第一个参数不包含softmax层的前向传播结果，第二个是训练数据的正确结果。
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=y,labels=tf.arg_max(y_,1)
    )
    #计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    #计算L2正则化损失
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    
    #计算模型的正则化损失，一般只计算神经网络边上权重的正则化损失，而不使用偏置项
    regularization = regularizer(weights1) + regularizer(weights2)
    
    #总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularization
    
    #设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,#基础衰减率
        global_step,#当前迭代轮数
        mnist.train.num_examples,
        LEARNING_RATE_DECAY#学习衰减率
    )
    
#使用梯度下降优化
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

#在训练神经网络模型时，每过一遍数据既需要通过反向传播来更新神经网络中的参数。
with tf.control_dependencies([train_step,variables_average_op]):
    train_op = tf.no_op(name='train')
    
#校验使用滑动平均模型的神经网络前向传播结果的正确是否正确
correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))

#这个运算首先将一个布尔型的数值转换为实数，然后计算平均值。这个平均值就是模型在这组数据上的正确性。
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#初始化会话并开始训练结果
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    #准备验证数据，一般在神经网络的训练过程中会通过验证数据来大致判断停止的条件和评判训练的效果
    validata_feed = {x:mnist.validation.images,
                    y_:mnist.validation.labels}
    test_feed = {x:mnist.test.images,
                y_:mnist.test.labels}
    #迭代训练神经
    for i in range(TRAINING_STEPS):
        if i % 1000 == 0:
            validation_acc = sess.run(accuracy,feed_dict=validata_feed)
            print('After %d training step(s),validation accuracy'
                 'using average model is %g '% (i,validation_acc))
            
        #产生这一轮使用的一个bacth的训练数据，并运行训练过程
        xs,ys = mnist.train.next_batch(BATCH_SIZE)
        sess.run(train_op,feed_dict={x:xs,y_:ys})
        
    #训练集结束后，在测试集上检测神经网络模型的最终正确性
    test_acc = sess.run(accuracy,feed_dict=test_feed)
    print ('After %d training step(s),test accuracy using average '
          'model is %g' % (TRAINING_STEPS,test_acc))


NameError: name 'learning_rate' is not defined

In [30]:
def main(argv=None):
    mnist = input_data.read_data_sets('/home/mj/Documents/data/MNIST_data',one_hot=True)
    train(mnist)
    
if __name__=='__main__':
    tf.app.run()

Extracting /home/mj/Documents/data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/mj/Documents/data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/mj/Documents/data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/mj/Documents/data/MNIST_data/t10k-labels-idx1-ubyte.gz


SystemExit: 

/home/mj/.conda/envs/ten/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
